In [23]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

#define size of chessboard
nx = 9
ny = 6

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
# Step through the list and search for chessboard corners
num=0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
       

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
        
cv2.destroyAllWindows()



#use the calibrate camera function with all the object and image points to learn...
#...the matrix and distortion coefficients
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

not saved camera_cal\calibration1.jpg
not saved camera_cal\calibration10.jpg
not saved camera_cal\calibration11.jpg
not saved camera_cal\calibration12.jpg
not saved camera_cal\calibration13.jpg
not saved camera_cal\calibration14.jpg
not saved camera_cal\calibration15.jpg
not saved camera_cal\calibration16.jpg
not saved camera_cal\calibration17.jpg
not saved camera_cal\calibration18.jpg
not saved camera_cal\calibration19.jpg
not saved camera_cal\calibration2.jpg
not saved camera_cal\calibration20.jpg
not saved camera_cal\calibration3.jpg
written camera_cal\calibration4.jpg
not saved camera_cal\calibration5.jpg
not saved camera_cal\calibration6.jpg


In [26]:
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
def undist(img):
    #use cv2.undistort()
      
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    return undist

#image1 = mpimg.imread('camera_cal/calibration1.jpg')
#cv2.imwrite('output_images/Camera_Calibration/distorted/calibration1.jpg', cv2.cvtColor(image1, cv2.COLOR_RGB2BGR))
            
#test for undistorted image
#undistorted = undist(image1)
#cv2.imwrite('output_images/Camera_Calibration/undistorted/calibration1_undist.jpg', (undistorted))
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(img1)
#ax1.set_title('Original Image', fontsize=50)
#ax2.imshow(undistorted)
#ax2.set_title('Undistorted Image', fontsize=50)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


True

In [52]:
# Define a function that applies Sobel x or y, 
# then takes an absolute value and applies a threshold.
# Note: calling your function with orient='x', thresh_min=20, thresh_max=100
# should produce output like the example image shown above this quiz.
def abs_sobel_thresh(img, orient, abs_thresh):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    
    if orient in ['x', 'X']:
    
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)

    elif orient in ['y', 'Y']:
    
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)

    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)

    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= abs_thresh[0]) & (scaled_sobel <= abs_thresh[1])] = 1

    # 6) Return this mask as your binary_output image
    return grad_binary

# Define a function that applies Sobel x and y, 
# then computes the magnitude of the gradient
# and applies a threshold
def mag_thresh(img, ksize, magnitude_thresh):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize)
    
    # 3) Calculate the magnitude 
    abs_sobel = np.sqrt((sobelx**2)+(sobely**2))
    
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    # 5) Create a binary mask where mag thresholds are met
    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel >= magnitude_thresh[0]) & (scaled_sobel <= magnitude_thresh[1])] = 1
    
    # 6) Return this mask as your binary_output image
    return mag_binary

# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(img, sobel_kernel, thresh):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, sobel_kernel)

    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)

    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    dir = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    dir_binary = np.zeros_like(dir)
    dir_binary[(dir >= thresh[0]) & (dir <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return dir_binary

def s_select(img, thresh):
    
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    
    # 2) Apply a threshold to the S channel
    S = hls[:,:,2]
    binary_output = np.zeros_like(S)
    binary_output[(S > thresh[0]) & (S <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output

def l_select(img, thresh):
    
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    
    # 2) Apply a threshold to the S channel
    S = hls[:,:,1]
    binary_output = np.zeros_like(S)
    binary_output[(S > thresh[0]) & (S <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output

def h_select(img, thresh):
    
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    
    # 2) Apply a threshold to the S channel
    S = hls[:,:,0]
    binary_output = np.zeros_like(S)
    binary_output[(S > thresh[0]) & (S <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output


#use HLS channel thresholding and Sobel operator to create binary image
def binaryimg (img1, hls_sthresh, sx_thresh, sy_thresh, sdir_thresh, magnitude_thresh, ksize):
    
    img = undist(img1)
    
    orient = 'x'
    gradx = abs_sobel_thresh(img, orient, sx_thresh)
    orient = 'y'
    grady = abs_sobel_thresh(img, orient, sy_thresh)
    mag_binary = mag_thresh(img, ksize, magnitude_thresh)
    dir_binary = dir_threshold(img, ksize, sdir_thresh)
    s_binary = s_select(img, hls_sthresh)
    h_binary = h_select(img, hls_hthresh)
    l_binary = l_select(img, hls_lthresh)


    combined = np.zeros_like(dir_binary)
    combined [((((h_binary==1) & (s_binary == 1))|(gradx == 1))|(((mag_binary == 1) & (grady == 1))& (dir_binary == 1)))]=1
    #combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))|(s_binary == 1)] = 1 
    #combined [((h_binary==1) & (s_binary == 1))] =1
    return combined


hls_sthresh = (170,255)
hls_hthresh = (10,60)
hls_lthresh = (20,255)
sx_thresh = (20, 30)
sy_thresh = (20, 90)
sdir_thresh = (0.7, 1.3)
magnitude_thresh = (30, 110)
ksize = 15



In [115]:
image = mpimg.imread('test_images/straight_lines1.jpg')
cv2.imwrite('output_images/Camera_Calibration/distorted/straight_lines1.jpg', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
cv2.imwrite('output_images/Camera_Calibration/undistorted/straight_lines1.jpg', cv2.cvtColor(undist(image), cv2.COLOR_BGR2RGB))

lt = (575, 460)
lb = (255,680)
rb = (1060, 680)
rt = (710, 460)

image = cv2.line(image, lb, lt, (255,0,0), 10)
image = cv2.line(image, lt, rt, (255,0,0), 10)
image = cv2.line(image, rt, rb, (255,0,0), 10)
#plt.imshow(image)

tlb = (200,image.shape[0])
tlt = (200, 0)
trb = ((image.shape[1]-250), image.shape[0])
trt = ((image.shape[1]-250), 0)

src = np.float32([[lt], [rt], [lb], [rb]])
dst = np.float32([[tlt], [trt], [tlb], [trb]])

Minv = cv2.getPerspectiveTransform(dst, src)
M = cv2.getPerspectiveTransform(src, dst)

def transfrm (img, src, dst):
    
    M = cv2.getPerspectiveTransform(src, dst)
    img_size = (img.shape[1], img.shape[0])
    
    ptransfrm = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return ptransfrm

binary = binaryimg(image, hls_sthresh, sx_thresh, sy_thresh, sdir_thresh, magnitude_thresh, ksize)
#binaryclr = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
#cv2.imwrite('output_images/binaryfinal.jpg', (binary))

#plt.imshow(binary, cmap='gray')


In [58]:
images = glob.glob('test_images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    binary = binaryimg(img, hls_sthresh, sx_thresh, sy_thresh, sdir_thresh, magnitude_thresh, ksize)
    binary = binary.astype('float32')
    binary = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
    
    binary = cv2.line(binary, lb, lt, (0,0,255), 1)
    binary = cv2.line(binary, lt, rt, (0,0,255), 1)
    binary = cv2.line(binary, rt, rb, (0,0,255), 1)
    
    trnsfrmd = transfrm(binary, src, dst)
    
    cv2.imshow('binary', binary)
    cv2.imshow('transformed',trnsfrmd)
    cv2.waitKey(0)
    #plt.imshow(cv2.cvtColor(binary, cv2.COLOR_BGR2RGB))  
cv2.destroyAllWindows()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [59]:
def visualizeLaneLines(warped, left_fit, right_fit, ploty, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    return newwarp

In [68]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    #print(leftx_base, rightx_base)

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        #(win_xleft_high,win_y_high),(0,255,0), 2) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),
        #(win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        #elif ((len(good_left_inds) < minpix) & (len(good_right_inds) > minpix)):
         #   leftx_current = leftx_current + np.int(np.mean(nonzerox[good_right_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        #elif ((len(good_right_inds) < minpix) & (len(good_left_inds) > minpix)):    
         #   rightx_current = rightx_current + np.int(np.mean(nonzerox[good_left_inds]))
            
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img, nonzero, nonzeroy, nonzerox, left_lane_inds, right_lane_inds, leftx_base, rightx_base

def appendLine(binary_warped, left_fit, right_fit, margin = 100):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
   
    minpoints = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    if ((len(leftx)<minpoints) |(len(lefty)<minpoints)|(len(rightx)<minpoints)|(len(righty)<minpoints)):
        appendattmpt = False
    else:
        appendattmpt = True
    return leftx, lefty, rightx, righty, appendattmpt

def fit_polynomial(leftx, lefty, rightx, righty):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Find our lane pixels first
    #print(leftx, rightx)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Fit a second order polynomial to each - convert to physical distance
    left_fit_m = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_m = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    return left_fit, right_fit, left_fit_m, right_fit_m


        
   
images = glob.glob('test_images/test5.jpg')
margin = 100

for fname in images:
    img = cv2.imread(fname)
    plt.clf()
    binary = binaryimg(img, hls_sthresh, sx_thresh, sy_thresh, sdir_thresh, magnitude_thresh, ksize)
    binary = binary.astype('float32')
    #binary = cv2.cvtColor(binary, cv2.COLOR_GRAY2RGB)
    
    trnsfrmd = transfrm(binary, src, dst)
    #print(trnsfrmd.shape)
    leftx, lefty, rightx, righty, out_img, nonzero, nonzeroy, nonzerox, left_lane_inds, right_lane_inds, leftx_base, rightx_base = find_lane_pixels(trnsfrmd)

    left_fit, right_fit, left_fit_m, right_fit_m = fit_polynomial(leftx, lefty, rightx, righty)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, out_img.shape[0]-1, out_img.shape[0] )
    y_eval = np.max(ploty)
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    window_img = np.zeros_like(out_img)
    
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    out_img = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    
    out_img = out_img.astype(np.uint8)

    #plt.imshow(out_img)
    
    newwarp = visualizeLaneLines (trnsfrmd, left_fit, right_fit, ploty, Minv)
    result = cv2.addWeighted(undist(img), 1, newwarp, 0.3, 0)
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

    #cv2.imshow('binary',binary)
    #cv2.waitKey(0)
        
cv2.destroyAllWindows()

In [69]:
def measure_curvature_real(left_fit_m, right_fit_m, y_eval):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1+(2*left_fit_m[0]*y_eval*ym_per_pix+left_fit_m[1])**2)**(3/2))/(np.absolute(2*left_fit_m[0]))  ## Implement the calculation of the left line here
    right_curverad = ((1+(2*right_fit_m[0]*y_eval*ym_per_pix+right_fit_m[1])**2)**(3/2))/(np.absolute(2*right_fit_m[0]))  ## Implement the calculation of the right line here
    
    CurveRad = abs((left_curverad+right_curverad)/2)
    
    sanitycheck = False
    
    if (((abs(left_curverad - right_curverad))<600) and (abs(left_curverad)>1000 and abs(right_curverad)>1000)):
        sanitycheck = True
    elif((left_curverad-right_curverad)<500):
        sanitycheck = True
    else:
        sanitycheck = False
    return CurveRad, sanitycheck, left_curverad, right_curverad

In [70]:
def measure_position_real(left_fit_m, right_fit_m, maxX, maxY):
    
    vehicleCenter = maxX/2

    #position at f(y=0) = Ay^2+ By + C
    #
    C0_l = left_fit_m[0]*maxY**2 + left_fit_m[1]*maxY + left_fit_m[2]
    C0_r = right_fit_m[0]*maxY**2 + right_fit_m[1]*maxY + right_fit_m[2]
    
    lanewidth = C0_r- C0_l
    
    Pos_err = ((C0_r- C0_l)/2 + C0_l) - vehicleCenter
    
    return Pos_err, lanewidth
    

In [72]:
images = glob.glob('test_images/*.jpg')
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension
num =0
for fname in images:
   
    img = cv2.imread(fname)
    binary = binaryimg(img, hls_sthresh, sx_thresh, sy_thresh, sdir_thresh, magnitude_thresh, ksize)
    binary = binary.astype('float32')
    #binary = cv2.cvtColor(binary, cv2.COLOR_GRAY2RGB)
    
    trnsfrmd = transfrm(binary, src, dst)
    leftx, lefty, rightx, righty, out_img, nonzero, nonzeroy, nonzerox, left_lane_inds, right_lane_inds, leftx_base, rightx_base = find_lane_pixels(trnsfrmd)
    left_fit, right_fit, left_fit_m, right_fit_m = fit_polynomial(leftx, lefty, rightx, righty)
    out_img = out_img.astype(np.uint8)
    
    maxX = out_img.shape[1]*xm_per_pix
    maxY = out_img.shape[0]*ym_per_pix
    
    CurveRad, sanitycheck, left_curverad, right_curverad = measure_curvature_real(left_fit_m, right_fit_m, y_eval)
    
    Pos_err, lanewidth = measure_position_real(left_fit_m, right_fit_m, maxX, maxY)
    print(images[num], "Curve: ", str.format('{0:.2f}', CurveRad) , "m position error:", str.format('{0:.2f}', Pos_err), "m")
    num = num+1
        
cv2.destroyAllWindows()

test_images\straight_lines1.jpg Curve:  5712.04 m position error: -0.14 m
test_images\straight_lines2.jpg Curve:  61974.41 m position error: -0.12 m
test_images\test1.jpg Curve:  1317.44 m position error: 0.02 m
test_images\test2.jpg Curve:  553.61 m position error: 0.13 m
test_images\test3.jpg Curve:  974.51 m position error: 0.02 m
test_images\test4.jpg Curve:  1238.34 m position error: 0.19 m
test_images\test5.jpg Curve:  850.28 m position error: -0.15 m
test_images\test6.jpg Curve:  880.09 m position error: 0.20 m


In [73]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from operator import add

In [110]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  
        
def pipeline (image):
    
    
    binary = binaryimg(image, hls_sthresh, sx_thresh, sy_thresh, sdir_thresh, magnitude_thresh, ksize)
    binary = binary.astype('float32')
    #cv2.imshow('binary',binary)
    trnsfrmd = transfrm(binary, src, dst)
    #cv2.imshow('trnsfrmd',trnsfrmd)
    appendattmpt = False
    Margin = 100
    
    if (leftlane.detected == True & rightlane.detected == True):
        print('append', leftlane.detected, rightlane.detected)
        leftx, lefty, rightx, righty, appendattmpt = appendLine(trnsfrmd, leftlane.current_fit, rightlane.current_fit, margin)
        left_fit, right_fit, left_fit_m, right_fit_m = fit_polynomial(leftx, lefty, rightx, righty)
        if (appendattmpt==False):
            leftx, lefty, rightx, righty, out_img, nonzero, nonzeroy, nonzerox, left_lane_inds, right_lane_inds, leftx_base, rightx_base = find_lane_pixels(trnsfrmd)
            left_fit, right_fit, left_fit_m, right_fit_m = fit_polynomial(leftx, lefty, rightx, righty)
            #leftlane.detected = True
            #rightlane.detected = True
                
    else:
        
        leftx, lefty, rightx, righty, out_img, nonzero, nonzeroy, nonzerox, left_lane_inds, right_lane_inds, leftx_base, rightx_base = find_lane_pixels(trnsfrmd)
        left_fit, right_fit, left_fit_m, right_fit_m = fit_polynomial(leftx, lefty, rightx, righty)
        #leftlane.detected = False
        rightlane.detected = False
        
    maxX = out_img.shape[1]*xm_per_pix
    maxY = out_img.shape[0]*ym_per_pix    
    CurveRad, curvsanitycheck, left_curverad, right_curverad = measure_curvature_real(left_fit_m, right_fit_m, y_eval)
    fitdiff=abs(left_fit[0]-right_fit[0])
    fitdiff1=abs(left_fit[1]-right_fit[1])
    #print(sanity check)
    Pos_err, lanewidth = measure_position_real(left_fit_m, right_fit_m, maxX, maxY)
    #print(fitdiff, fitdiff1, curvsanitycheck, lanewidth)
    if (curvsanitycheck == True and (4.1<lanewidth<4.5) and (fitdiff<0.00035) and (fitdiff1<0.2)): #fitdiff<0.00019 fitdiff1<0.13
        #print('curv Sanity Check Passed Left Curve Rad:', left_curverad, 'right Curve Rad: ', right_curverad, 'lanewidth', lanewidth)
        
        if (leftlane.detected == True):
            #print('averaging polycoef')
            leftlane.best_fit = np.add(leftlane.best_fit, left_fit)/2
            rightlane.best_fit = np.add(rightlane.best_fit, right_fit)/2
            leftlane.current_fit = left_fit
            rightlane.current_fit = right_fit
            leftlane.radius_of_curvature =  left_curverad
            rightlane.radius_of_curvature = right_curverad
            leftlane.line_base_pos = Pos_err
            rightlane.line_base_pos = Pos_err
            
        
        else:
            leftlane.best_fit = left_fit
            rightlane.best_fit = right_fit
            leftlane.current_fit = left_fit
            rightlane.current_fit = right_fit
            leftlane.radius_of_curvature =  left_curverad
            rightlane.radius_of_curvature = right_curverad
            leftlane.line_base_pos = Pos_err
            rightlane.line_base_pos = Pos_err
            #print('initializing polycoeff')
            leftlane.detected = True
        
        
    else:
        #print('curv Sanity Check Failed Left Curve Rad:', left_curverad, 'right Curve Rad: ', right_curverad, 'lanewidth', lanewidth)

        left_fit = leftlane.current_fit
        right_fit = rightlane.current_fit
        right_curverad = rightlane.radius_of_curvature
        Pos_err = rightlane.line_base_pos
        left_curverad = leftlane.radius_of_curvature
        CurveRad = abs((left_curverad+right_curverad)/2)

    Minv = cv2.getPerspectiveTransform(dst, src)
    ploty = np.linspace(0, trnsfrmd.shape[0]-1, trnsfrmd.shape[0] )
    newwarp = visualizeLaneLines (trnsfrmd, left_fit, right_fit, ploty, Minv)
    #cv2.imshow('newwarp',newwarp)
    result = cv2.addWeighted(undist(image), 1, newwarp, 0.3, 0)
   

    if Pos_err > 0:
        message = '{:.2f} m left'.format(Pos_err)
    else:
        message = '{:.2f} m right'.format(-Pos_err)
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontColor = (255, 255, 255)
    cv2.putText(result, 'Road Curvature: {:.0f} m'.format(CurveRad), (250, 75), font, 2, fontColor, 2)
    cv2.putText(result, 'Vehicle is {} of center'.format(message), (150, 150), font, 2, fontColor, 2)
    #cv2.imshow('result',result)
    return result

In [101]:
image = mpimg.imread('test_images/test3.jpg')
plt.clf()
leftlane = Line()
rightlane = Line()
result = pipeline(image)
plt.imshow(result)

In [111]:
output = 'output_images/project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
leftlane = Line()
rightlane = Line()
#clip1 = VideoFileClip('project_video.mp4').subclip(16,26)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)

Moviepy - Building video output_images/project_video_output.mp4.
Moviepy - Writing video output_images/project_video_output.mp4



Moviepy - Done !
Moviepy - video ready output_images/project_video_output.mp4
Wall time: 6min 3s


In [112]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))